In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

### Preparo y entreno el label encoder y el one hot encoder con la info de todos los avisos existentes

In [2]:
avisos_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [3]:
avisos_totales['nombre_area'].value_counts().size

188

- Label encoding para las 188 areas, y entreno el One Hot Encoding con esos 188 codigos (0-187)

In [4]:
#Fundamental usar para el label encoder una columna DF que contenga TODOS los posibles valores categoricos del nombre de area, es decir, lo saco del DF de TODOS los anuncios.
#Esto es para que si en un set no estan todos, igual los codifique de la misma forma. SI lo entreno con un set que no tiene todos, al ingresar un set con valores nuevos
#va a tirar error porque no tenia esos labels entrenados

#UNA VEZ ENTRENADO EL LABEL ENCODER, Y EL ONEHOTENCODER, USAR SIEMRE EL MISMO PARA EL RESTO DEL NOTEBOOK CODIFICAR LOS SETS DE DATA (TEST Y TRAIN) Y EL SET DE TEST100K

le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


### Cargo historial de postulaciones x usuario x area, ya encodeado con One Hot Encoding

In [8]:
users_ids_history = pd.read_csv('historial_cant_post_xuser_xarea.csv', dtype={i+1:'int8' for i in range(188)})
users_ids_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346764 entries, 0 to 346763
Columns: 189 entries, idpostulante to 187
dtypes: int8(188), object(1)
memory usage: 64.8+ MB


In [ ]:
#users_ids_history.to_csv('historial_cant_post_xuser_xarea.csv', index=False)

### Cargo Dataset y test100k para operar

In [9]:
data_set_complete_v2 = pd.read_csv('data_set_complete_v2.csv', usecols=['sepostulo','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente'],\
                                dtype={'sepostulo':'int8', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8'})

data_set_complete_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11870121 entries, 0 to 11870120
Data columns (total 19 columns):
sepostulo           int8
idpostulante        object
veces_visto         int16
sexo                int8
edad                int8
titulo              object
nombre_area         category
estudio_encurso     int8
estudio_graduado    int8
tipo_full-time      int8
tipo_part-time      int8
tipo_otro           int8
zona_gba            int8
zona_caba           int8
nivel_senior        int8
nivel_junior        int8
nivel_otro          int8
nivel_jefe          int8
nivel_gerente       int8
dtypes: category(1), int16(1), int8(15), object(2)
memory usage: 396.2+ MB


In [10]:
#Achico a 1 millon para maniabrabilidad en terminos de performance
data = data_set_complete_v2.sample(1000000)

del data_set_complete_v2

data.reset_index(drop=True, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
sepostulo           1000000 non-null int8
idpostulante        1000000 non-null object
veces_visto         1000000 non-null int16
sexo                1000000 non-null int8
edad                1000000 non-null int8
titulo              1000000 non-null object
nombre_area         1000000 non-null category
estudio_encurso     1000000 non-null int8
estudio_graduado    1000000 non-null int8
tipo_full-time      1000000 non-null int8
tipo_part-time      1000000 non-null int8
tipo_otro           1000000 non-null int8
zona_gba            1000000 non-null int8
zona_caba           1000000 non-null int8
nivel_senior        1000000 non-null int8
nivel_junior        1000000 non-null int8
nivel_otro          1000000 non-null int8
nivel_jefe          1000000 non-null int8
nivel_gerente       1000000 non-null int8
dtypes: category(1), int16(1), int8(15), object(2)
memory usage: 33.4+ MB


In [11]:
test100k = pd.read_csv('test100k_complete_v2.csv', usecols=['id','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente'],\
                                dtype={'id':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8'})

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
id                  100000 non-null int64
idpostulante        100000 non-null object
veces_visto         100000 non-null int16
sexo                100000 non-null int8
edad                100000 non-null int8
titulo              100000 non-null object
nombre_area         100000 non-null category
estudio_encurso     100000 non-null int8
estudio_graduado    100000 non-null int8
tipo_full-time      100000 non-null int8
tipo_part-time      100000 non-null int8
tipo_otro           100000 non-null int8
zona_gba            100000 non-null int8
zona_caba           100000 non-null int8
nivel_senior        100000 non-null int8
nivel_junior        100000 non-null int8
nivel_otro          100000 non-null int8
nivel_jefe          100000 non-null int8
nivel_gerente       100000 non-null int8
dtypes: category(1), int16(1), int64(1), int8(14), object(2)
memory usage: 4.0+ MB


### Uso label encoding para nombre_area

In [12]:
data['nombre_area'] = le.transform(data['nombre_area'].astype(str))

In [13]:
test100k['nombre_area'] = le.transform(test100k['nombre_area'].astype(str))

### One Hot Encoding para nombre_area

In [14]:
encoded_areas = enc.transform(data['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

data = data.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 207 entries, sepostulo to 187
dtypes: int16(1), int64(1), int8(203), object(2)
memory usage: 218.4+ MB


In [15]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,Y1A13w,0,1,34,Musimundo SAN MIGUEL necesita Vendedores,185,0,3,1,...,0,0,0,0,0,0,0,1,0,0
1,1,xkPzJz0,0,0,32,P- Abogado,115,0,4,1,...,0,0,0,0,0,0,0,0,0,0
2,1,YL18pP,0,1,28,Administrativo de Ventas,185,0,3,1,...,0,0,0,0,0,0,0,1,0,0
3,1,EOd329,0,0,34,M - Peones y Ayudantes de cocina,82,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,EJBNmo,0,0,30,Operario Z/ General Rodríguez,143,2,4,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
encoded_areas = enc.transform(test100k['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

test100k = test100k.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 207 entries, id to 187
dtypes: int16(1), int64(2), int8(202), object(2)
memory usage: 22.5+ MB


In [17]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


### Merge del historial con One Hot Coding

In [18]:
# Aca meto la info del historial (matriz de 188 con el one hot encoding)
data = data.merge((data['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int8', copy=False)), left_index=True, right_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 395 entries, sepostulo to 187
dtypes: int16(1), int64(1), int8(391), object(2)
memory usage: 397.7+ MB


In [19]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,Y1A13w,0,1,34,Musimundo SAN MIGUEL necesita Vendedores,185,0,3,1,...,0,0,0,0,0,0,0,1,0,0
1,1,xkPzJz0,0,0,32,P- Abogado,115,0,4,1,...,0,0,0,0,0,0,0,0,0,0
2,1,YL18pP,0,1,28,Administrativo de Ventas,185,0,3,1,...,0,0,0,0,0,0,0,70,1,0
3,1,EOd329,0,0,34,M - Peones y Ayudantes de cocina,82,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,EJBNmo,0,0,30,Operario Z/ General Rodríguez,143,2,4,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Aca meto la info del historial (matriz de 188 con el one hot encoding)
test100k = test100k.merge((test100k['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int8', copy=False)), left_index=True, right_index=True)

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 395 entries, id to 187
dtypes: int16(1), int64(2), int8(390), object(2)
memory usage: 40.4+ MB


In [21]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,4,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


### Pruebo con Random Forest

In [22]:
#Saco edad porque vimos que resta score
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data['sepostulo'], test_size=0.3)

In [27]:
clf = RandomForestClassifier(n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf.score(test_set, test_classes), 'Entrenamiento:', clf.score(train_set, train_classes))

Test: 0.8865466666666667 Entrenamiento: 0.9016171428571429


- Con historial "fixeado"
- Test: 0.8865466666666667 Entrenamiento: 0.9016171428571429 (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
- Test: 0.88909 Entrenamiento: 0.9064142857142857 (n_estimators=40, min_samples_leaf=6, min_samples_split=5, n_jobs=-1)   Kaggle: 0.87545
- Test: 0.88608 Entrenamiento: 0.90129 (n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)

- Con historiales negativos:
- Test: 0.8863633333333333 Entrenamiento: 0.9017185714285715
- Kaggle: 0.86702

In [25]:
test100k['sepostulo'] = clf.predict(test100k.drop(columns=['edad','id','idpostulante', 'titulo']))

In [26]:
test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_historial_arreglado.csv', index=False)

In [31]:
clf2 = tree.DecisionTreeClassifier().fit(train_set, train_classes)
print('Test:', clf2.score(test_set, test_classes), 'Entrenamiento:', clf2.score(train_set, train_classes))

Test: 0.8704633333333334 Entrenamiento: 0.9947728571428571


- Test: 0.8704633333333334 Entrenamiento: 0.9947728571428571
- Kaggle: 0.84400

In [32]:
test100k['sepostulo'] = clf2.predict(test100k.drop(columns=['edad','sepostulo','id','idpostulante', 'titulo']))

In [34]:
test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_NewParameters_GridS_DecisTree.csv', index=False)

### Veamos como funciona para muestras mas pequenias, para ver si podemos reducir en busqueda de un aumento de velocidad de procesamiento, sin perder accuracy. Si tomamos como que test100k fuese nuestro set de test, y como usualmente se usa el 30% para test, entonces el total de train+test seria 340 mil

In [23]:
data_small = data.sample(350000)
data_small.reset_index(drop=True, inplace=True)

In [24]:
del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data_small.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data_small['sepostulo'], test_size=0.3)

In [28]:
clf3 = RandomForestClassifier(n_estimators=30, min_samples_leaf=5, min_samples_split=4, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf3.score(test_set, test_classes), 'Entrenamiento:', clf3.score(train_set, train_classes))

Test: 0.876752380952381 Entrenamiento: 0.9021428571428571


- Test: 0.876752380952381 Entrenamiento: 0.9021428571428571 (n_estimators=30, min_samples_leaf=5, min_samples_split=4, n_jobs=-1)

In [29]:
#GRID SEARCH
parameters = {'min_samples_leaf':[6,7,8,9], 'min_samples_split':[3,4,5,6]}
rndforst = RandomForestClassifier(n_estimators=30, n_jobs=-1)
clf3GS =  GridSearchCV(rndforst, parameters)
clf3GS.fit(train_set, train_classes)
clf3GS.best_params_ 

{'min_samples_leaf': 6, 'min_samples_split': 5}

In [30]:
print('Test:', clf3GS.score(test_set, test_classes), 'Entrenamiento:', clf3GS.score(train_set, train_classes))

Test: 0.8759047619047619 Entrenamiento: 0.8957755102040816


- Test: 0.8759047619047619 Entrenamiento: 0.8957755102040816
- Kaggle: 0.86963

In [32]:
test100k['sepostulo'] = clf3GS.predict(test100k.drop(columns=['edad', 'sepostulo','id','idpostulante', 'titulo']))

In [33]:
test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_350msamples_gridserach.csv', index=False)

### Vemos que da scores un poco inferiores. Veamos entonces con muestras mas grandes (se corre nuevamente el kernel con el sample inicial de 2 millones)

In [20]:
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data['sepostulo'], test_size=0.3)

In [21]:
clf4 = RandomForestClassifier(n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf4.score(test_set, test_classes), 'Entrenamiento:', clf4.score(train_set, train_classes))

Test: 0.892795 Entrenamiento: 0.9068257142857142


- Test: 0.892795 Entrenamiento: 0.9068257142857142
- Kaggle: 0.87659

In [22]:
test100k['sepostulo'] = clf4.predict(test100k.drop(columns=['edad', 'id','idpostulante', 'titulo']))

In [23]:
test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_RndTree_2Msamples.csv', index=False)

### Veamos entonces con muestras mas grandes (se corre nuevamente el kernel con el sample inicial de 3 millones)

In [19]:
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data['sepostulo'], test_size=0.3)

In [20]:
clf5 = RandomForestClassifier(n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf5.score(test_set, test_classes), 'Entrenamiento:', clf5.score(train_set, train_classes))

Test: 0.8969855555555556 Entrenamiento: 0.9096571428571428


- Test: 0.8969855555555556 Entrenamiento: 0.9096571428571428
- Kaggle: 0.88024

In [21]:
test100k['sepostulo'] = clf5.predict(test100k.drop(columns=['edad', 'id','idpostulante', 'titulo']))

In [22]:
test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_RndTree_3Msamples.csv', index=False)